# **YOLOv8 TRAINING**

## **Initial configuration**

**Install YOLOv8**
**From pip**</br>
`pip install "ultralytics<=8.3.40"`

In [ ]:
#!pip install "ultralytics<=8.3.40"
#!pip install ultralytics==8.2.103
#!pip install roboflow==1.1.48
#!pip install comet_ml==3.47.4

**Import libraries**

In [3]:
import comet_ml
import os
import roboflow
import urllib.request
import yaml

from ultralytics import YOLO

In [4]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.103 🚀 Python-3.12.7 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4034MiB)
Setup complete ✅ (8 CPUs, 7.6 GB RAM, 60.3/294.2 GB disk)


**Operating system features**

In [5]:
!cat /etc/os-release

PRETTY_NAME="Ubuntu 22.04.5 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.5 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


**Graphics card characteristics**

In [6]:
!nvidia-smi

Mon Jan 13 23:03:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1050 Ti     Off |   00000000:01:00.0 Off |                  N/A |
| N/A   39C    P8             N/A / ERR!  |       4MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce GTX 1050 Ti (UUID: GPU-b9a2dc58-db69-5164-1b51-3c9ce1e254bb)


**Environment configuration**

In [8]:
# Run local o not
PROJ_ROOT = os.getcwd()
local = True if os.path.basename(PROJ_ROOT) == 'notebooks' else False

# Directories
if not local:
  os.makedirs('data/processed', exist_ok=True)
  os.makedirs('models', exist_ok=True)
  data_relative_path ='data'
  models_relative_path = 'models'
else:
  data_relative_path ='../data'
  models_relative_path = '../models'

DATA_DIR = os.path.join(PROJ_ROOT, data_relative_path)
MODELS_DIR = os.path.join(PROJ_ROOT, models_relative_path)
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, 'processed')

In [14]:
class CFG:
  DEBUG = True
  FRACTION = 0.01 if DEBUG else 1.0

  # Model
  MODEL_SIZE = "n"
  PRETRAINED = True

  # Clases
  CLASSES = ['hard_hat',
            'no_hard_hat',
            'no_safety_harness',
            'no_safety_vest',
            'person',
            'safety_harness',
            'safety_vest']
  NUM_CLASSES_TO_TRAIN = len(CLASSES)

  # Training
  EPOCHS = 4 if DEBUG else 500

  # Dataset
  DATASET_VERSION = 5
  DATASET_NAME = f"ppe_dataset_v{DATASET_VERSION}"
  DATASET_PATH = os.path.join(PROCESSED_DATA_DIR, DATASET_NAME)

  # Output dir
  OUTPUT_DIR = MODELS_DIR

if not CFG.DEBUG:
  print('\033[34m INFO: This will take several minutes! \033[0m')
else:
  print('\033[93m WARNING: Configuration is set on DEBUG! \033[0m')

## **Import dataset**

In [7]:
# Login into roboflow
roboflow.login()

rf = roboflow.Roboflow()

project = rf.workspace("deeplearning-cwudo").project("yolo_ppe_detection")
version = project.version(CFG.DATASET_VERSION)
dataset = version.download(model_format="yolov8", location=CFG.DATASET_PATH, overwrite=True)

In [4]:
# Modify data yaml file
def modify_yaml_file(file_path):
  with open(file_path, 'r+') as file:
    try:
      data = yaml.safe_load(file)
      file.seek(0)
      file.truncate(0)

      data["train"] = os.path.join(CFG.DATASET_PATH, "train/images")
      data["val"] = os.path.join(CFG.DATASET_PATH, "valid/images")
      data["test"] = os.path.join(CFG.DATASET_PATH, "test/images")

      yaml.dump(data, file)
      yaml_data = yaml.dump(data, default_style=False)
      print(yaml_data)

    except yaml.YAMLError as e:
      print('Error reading YAML: ', e)

  file.close()

data_file_path = os.path.join(CFG.DATASET_PATH, 'data.yaml')
modify_yaml_file(data_file_path)

names:
- hard_hat
- no_hard_hat
- no_safety_harness
- no_safety_vest
- person
- safety_harness
- safety_vest
nc: 7
roboflow:
  license: CC BY 4.0
  project: yolo_ppe_detection
  url: https://universe.roboflow.com/deeplearning-cwudo/yolo_ppe_detection/dataset/1
  version: 1
  workspace: deeplearning-cwudo
test: /home/harvey/Documents/yolo_ppe_detection/notebooks/../data/processed/ppe_dataset_v1/test/images
train: /home/harvey/Documents/yolo_ppe_detection/notebooks/../data/processed/ppe_dataset_v1/train/images
val: /home/harvey/Documents/yolo_ppe_detection/notebooks/../data/processed/ppe_dataset_v1/valid/images



## **Training**

**Download YOLOv8 base models**

In [10]:
def donwload_base_model(model_size = "n", pretrained = True):
  base_models_dir_path = os.path.join(CFG.OUTPUT_DIR, 'base')
  os.makedirs(base_models_dir_path, exist_ok=True)

  model_ext = "pt" if pretrained else "yaml"
  model_name = f"yolov8{model_size}.{model_ext}"

  model_file = os.path.join(base_models_dir_path, model_name)
  if not pretrained:
    model_file = model_name
    print(f"Empty {model_file} was charged.")

  else:
    if model_name not in os.listdir(base_models_dir_path):
      print(f'Downloading {model_name} ...')
      try:
        urllib.request.urlretrieve(f"https://github.com/ultralytics/assets/releases/download/v8.2.0/{model_name}", model_file)
      except:
        print(f'Something went wrong with downloading ...')
    else:
      print(f'The model {model_name} already exists!')
  return model_file, model_name

model_file, model_name = donwload_base_model(model_size=CFG.MODEL_SIZE, pretrained=CFG.PRETRAINED)

The model yolov8n.pt already exists!


**Project configuration**

In [11]:
# Project name conf
model_name_splited = model_name.split('.')
project_name = f"train_{model_name_splited[0]}_ppe_detection{'_debug' if CFG.DEBUG else ''}"
project_path = os.path.join(CFG.OUTPUT_DIR, project_name)
train_name = f"{model_name_splited[1]}_{CFG.EPOCHS}_epochs"
print("Project name:\t\t", project_name)
print("Project path:\t\t", project_path)
print("Train name:\t\t", train_name)
print("Model name:\t\t", model_name)

Project name:		 train_yolov8n_ppe_detection_debug
Project path:		 /home/harvey/Documents/yolo_ppe_detection/notebooks/../models/train_yolov8n_ppe_detection_debug
Train name:		 pt_10_epochs
Model name:		 yolov8n.pt


**Commet ML configuration**

In [ ]:
# Commet config
comet_ml.login()
experiment_config = comet_ml.ExperimentConfig(name=train_name)
exp = comet_ml.start(project_name=project_name, experiment_config=experiment_config)

**Training configuration**

In [25]:
cfg_train = {
  "data": data_file_path,
  "epochs": CFG.EPOCHS,
  "patience": 50,
  "batch": -1,
  "imgsz": 640,
  "save": True,
  "save_period": -1,
  "cache": True,
  "device": [0],
  "project": project_path, # kaggle -> project_name, colab -> project_path
  "name": train_name,
  "exist_ok": False,
  "pretrained": True,
  "optimizer": "auto",
  "resume": False,
  "fraction": CFG.FRACTION,
  "profile": False,
  "lr0": 0.01,
  "lrf": 0.01,
  "momentum": 0.937,
  "val": True,
  "iou": 0.8,
  "augment": True,
  "plots": True
}

for key, value in cfg_train.items():
    print(key, ":", value)

epochs : 3
patience : 50
batch : 32
imgsz : 640
save : True
save_period : -1
cache : True
device : [0, 1]
project : /home/harvey/Documents/yolo_ppe_detection/notebooks/../models/train_yolov8n_ppe_detection_test
name : pt_3_epochs
exist_ok : False
pretrained : True
optimizer : auto
resume : False
fraction : 0.01
profile : False
lr0 : 0.01
lrf : 0.01
momentum : 0.937
val : True
iou : 0.8
augment : True
plots : True


**Load the model**

In [26]:
model = YOLO(model_file)

**Model information**

In [27]:
model.info()

YOLOv8n summary: 225 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs


(225, 3157200, 0, 8.8575488)

**Model training**

In [ ]:
results = model.train(**cfg_train)
exp.end()

New https://pypi.org/project/ultralytics/8.3.49 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.103 🚀 Python-3.12.7 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4034MiB)
engine/trainer: task=detect, mode=train, model=/home/harvey/Documents/yolo_ppe_detection/notebooks/../models/base/yolov8n.pt, data=/home/harvey/Documents/yolo_ppe_detection/notebooks/../data/processed/ppe_dataset_v1/data.yaml, epochs=3, time=None, patience=20, batch=8, imgsz=640, save=True, save_period=-1, cache=True, device=0, workers=8, project=/home/harvey/Documents/yolo_ppe_detection/notebooks/../models/yolov8n_ppe_detection_test, name=pt_3_epochs2, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=0.01, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=Fal

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/alexpfoccori/home-harvey-documents-yolo-ppe-detection-notebooks-models-yolov8n-ppe-detection-test/d919170f85d74236b9b583d01c12928e



Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/harvey/Documents/yolo_ppe_detection/data/processed/ppe_dataset_v1/train/labels.cache... 18 images, 0 backgrounds, 0 corrupt: 100%|██████████| 18/18 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...

                   all        520       4287     0.0096      0.306     0.0455     0.0278

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        2/3      1.18G      1.538      3.634      1.813         26        640: 100%|██████████| 3/3 [00:02<00:00,  1.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:08<00:00,  3.84it/s]


                   all        520       4287     0.0128      0.218     0.0711     0.0409

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        3/3      1.22G      1.327      3.193        1.6         27        640: 100%|██████████| 3/3 [00:00<00:00,  3.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:08<00:00,  3.78it/s]


                   all        520       4287    0.00157      0.203     0.0358     0.0153

3 epochs completed in 0.016 hours.
Optimizer stripped from /home/harvey/Documents/yolo_ppe_detection/notebooks/../models/yolov8n_ppe_detection_test/pt_3_epochs2/weights/last.pt, 6.2MB


## **Train Results**

**Import libraries**

In [ ]:
import glob
import matplotlib.pyplot as plt
import pandas as pd

from PIL import Image

**Results**

In [ ]:
result_path = os.path.join(model.overrides.get("project"), model.overrides.get("name"))
result_path

**Images of the results**

In [ ]:
image_files = [
  i for i in
  glob.glob(f"{result_path}/*.png") + glob.glob(f"{result_path}/*.jpg")
  if "batch" not in i
]

image_files

**Plot images of the results**

In [ ]:
for image in sorted(image_files):
  image_name = image.split("/")[-1]
  print(image_name)

  img = Image.open(image)
  plt.imshow(img)
  plt.axis("off")
  plt.show()

**Read CSV result**

In [ ]:
df = pd.read_csv(f"{result_path}/results.csv")
df = df.rename(columns=lambda x: x.replace(" ", ""))
df